## 2

#### (a)

In [ ]:
!pip install yfinance -q

     |████████████████████████████████| 62 kB 1.5 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('constituents.csv')
data
print("Shape of data: ", data.shape)
symbol_list = data.Symbol.tolist()

Shape of data:  (505, 3)


In [ ]:
import pandas_datareader.data as web
import yfinance as yf
yf.pdr_override()

start_date = '2012-06-11'
end_date = '2022-06-09'
df = web.get_data_yahoo(symbol_list, start=start_date, end = end_date)

[*********************100%***********************]  505 of 505 completed

15 Failed downloads:
- DD: No data found for this date range, symbol may be delisted
- ANTM: No data found, symbol may be delisted
- BLL: No data found, symbol may be delisted
- FB: No data found, symbol may be delisted
- DISCK: No data found, symbol may be delisted
- XLNX: No data found, symbol may be delisted
- DISCA: No data found, symbol may be delisted
- CERN: No data found, symbol may be delisted
- INFO: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- KSU: No data found, symbol may be delisted
- PBCT: No data found, symbol may be delisted
- WLTW: No data found, symbol may be delisted
- VIAC: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [ ]:
adj_close_data = df['Adj Close']
print("Shape of Data: ", adj_close_data.shape)

Shape of Data:  (2516, 505)


In [ ]:
start = '2012-06-12'
end = '2022-06-09'
close = adj_close_data[(adj_close_data.index >= start) & (adj_close_data.index <= end)]
close = close[['AAPL', 'GOOG', 'TSLA']]

In [ ]:
df.head()

Adj Close                                                   \
                    A        AAL        AAP       AAPL ABBV        ABC   
Date                                                                     
2012-06-11  25.421358  10.709542  67.725334  17.417259  NaN  31.003254   
2012-06-12  25.740919  10.775536  68.832703  17.569418  NaN  31.462427   
2012-06-13  25.434402  11.284616  66.195740  17.447445  NaN  30.994751   
2012-06-14  25.662659  11.765414  66.308372  17.428232  NaN  31.428419   
2012-06-15  26.093084  11.341179  65.445023  17.507515  NaN  31.709030   

                                                        ...   Volume       \
                 ABMD        ABT        ACN       ADBE  ...      XEL XLNX   
Date                                                    ...                 
2012-06-11  21.190001  24.169592  47.331696  31.129999  ...  9478600  NaN   
2012-06-12  21.450001  24.346411  47.430946  31.770000  ...  3333500  NaN   
2012-06-13  20.950001  24.169592  47.083591  31.790001  ...  2429600  NaN   
2012-06-14  21.420000  24.417143  47.836205  31.670000  ...  2029200  NaN   
2012-06-15  22.150000  24.558598  48.861744  32.389999  ...  3033500  NaN   

                                                                               
                 XOM     XRAY      XYL      YUM      ZBH    ZBRA     ZION ZTS  
Date                                                                           
2012-06-11  18031800   559800   816100  7750374   762200  314600  2794100 NaN  
2012-06-12  14162100   629000  1222700  6142795   684023  195700  2418100 NaN  
2012-06-13  15098100   663200   891300  4663328   663835  203300  1932100 NaN  
2012-06-14  16942300   845200  1478400  5460649   994980  192000  2060200 NaN  
2012-06-15  27342800  1386500  3023700  4794499  3389524  336400  3039100 NaN  

[5 rows x 3030 columns]

In [ ]:
daily_prices_return = close.pct_change().dropna()
daily_prices_return.head()

,AAPL,GOOG,TSLA
Date,,,
2012-06-13,-0.006942,-0.007096,0.003709
2012-06-14,-0.001101,-0.003636,-0.012765
2012-06-15,0.004549,0.009767,0.017693
2012-06-18,0.020292,0.011231,0.064527
2012-06-19,0.002782,0.018709,0.007851


#### (b)

In [ ]:
periods_per_year = 252
n_years = daily_prices_return.shape[0] / periods_per_year
annualized_return = np.power(np.prod(1+daily_prices_return), (1/n_years)) - 1
annualized_vol = daily_prices_return.std() / np.sqrt(periods_per_year)
MDD = (close.max() - close.min()) / close.max() * 100

In [ ]:
print(annualized_return)
print(annualized_vol)
print(MDD)

AAPL    0.237939
GOOG    0.236752
TSLA    0.619004
dtype: float64
AAPL    0.001134
GOOG    0.001027
TSLA    0.002237
dtype: float64
AAPL    99.448306
GOOG    99.336470
TSLA    99.754519
dtype: float64


In [ ]:
def start_end_of_drawdown(close):
  start_list = []
  end_list = []
  for column in close.columns:
    high = close[column][(close == close.max())[column]].index[0]
    low = close[column][(close == close.min())[column]].index[0]
    start, end = sorted([high, low])
    start_list.append(start)
    end_list.append(end)
  start = pd.Series(start_list, index=close.columns)
  end = pd.Series(end_list, index=close.columns)
  return start, end

In [ ]:
start, end = start_end_of_drawdown(close)

#### (C)

In [ ]:
start, end = '2012-06-12', '2022-06-09'
high_data, low_data = df['High'], df['Low']
high = high_data[(high_data.index >= start) & (high_data.index <= end)]
high = high[['AAPL', 'GOOG', 'TSLA']]
low = low_data[(low_data.index >= start) & (low_data.index <= end)]
low = low[['AAPL', 'GOOG', 'TSLA']]

In [ ]:
def get_high_lows_lookback(high, low, lookback_days):
    high_df = high.rolling(lookback_days).max().shift(1)
    low_df = low.rolling(lookback_days).min().shift(1)
    return high_df, low_df

In [ ]:
lookback_days = 50
lookback_high, lookback_low = get_high_lows_lookback(high, low, lookback_days)

In [ ]:
def get_long_short(close, lookback_high, lookback_low):
    return_df = close.copy()
    return_df[:] = 0
    return_df[close > lookback_high] = 1
    return_df[close < lookback_low] = -1
    return_df = return_df.astype('int64')
    return return_df

In [ ]:
signal = get_long_short(close, lookback_high, lookback_low)

In [ ]:
def clear_signals(signals, window_size):
    # Start with buffer of window size
    # This handles the edge case of calculating past_signal in the beginning
    clean_signals = [0]*window_size
    
    for signal_i, current_signal in enumerate(signals):
        # Check if there was a signal in the past window_size of days
        has_past_signal = bool(sum(clean_signals[signal_i:signal_i+window_size]))
        # Use the current signal if there's no past signal, else 0/False
        clean_signals.append(not has_past_signal and current_signal)
        
    # Remove buffer
    clean_signals = clean_signals[window_size:]

    # Return the signals as a Series of Ints
    return pd.Series(np.array(clean_signals).astype(np.int), signals.index)


def filter_signals(signal, lookahead_days):
    columns = signal.columns
    return_df = pd.DataFrame()
    for column in columns:
        data = signal[column]
        positive_signal = data.copy()
        negative_signal = data.copy()
        positive_signal[positive_signal < 0] = 0
        negative_signal[negative_signal > 0] = 0
        cleared_positive_signal = clear_signals(positive_signal, lookahead_days)
        cleared_negative_signal = clear_signals(negative_signal*-1, lookahead_days)
        cleared_signal = cleared_positive_signal + cleared_negative_signal * (-1)
        return_df[column] = cleared_signal
    return_df = return_df.astype('int64')
    return return_df

In [ ]:
filtered_signal = filter_signals(signal, 5)
filtered_signal

,AAPL,GOOG,TSLA
Date,,,
2012-06-12,0,0,0
2012-06-13,0,0,0
2012-06-14,0,0,0
2012-06-15,0,0,0
2012-06-18,0,0,0
...,...,...,...
2022-06-02,0,0,0
2022-06-03,0,0,0
2022-06-06,0,0,0


### 3

#### (a)

In [ ]:
# 한 달 단위로 모멘텀 계산
close

,AAPL,GOOG,TSLA
Date,,,
2012-06-12,17.569418,14.074744,1.977333
2012-06-13,17.447445,13.974869,1.984667
2012-06-14,17.428232,13.924059,1.959333
2012-06-15,17.507515,14.060049,1.994000
2012-06-18,17.862776,14.217958,2.122667
...,...,...,...
2022-06-02,151.000259,117.746002,258.333344
2022-06-03,145.178345,114.564003,234.516663
2022-06-06,145.937286,117.010498,238.279999


#### (b)

#### (c)

### 4

(a)

In [ ]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next
class Solution:
    def reverseList(self, head: ListNode) -> ListNode:
        prev, node = None, head
        while node:
            nxt = node.next
            node.next = prev
            prev = node
            node = nxt
        return prev

(b)

In [ ]:
class Solution:
    def generate(self, numRows: int) -> List[List[int]]:
        A = [[1]]
        for idx in range(numRows-1):
            length = idx + 2
            new_list = []
            new_list.append(1)
            for i in range(1,length-1):
                value = A[idx][i-1] + A[idx][i]
                new_list.append(value)
            new_list.append(1)
            A.append(new_list)
        return A

(c)

In [ ]:
class Solution:
    def oddEvenList(self, head: ListNode) -> ListNode:
        if not head: 
            return head
        odd = head
        even = head.next
        evenHead = even
        while(even and odd and even.next and odd.next):
            odd.next = even.next
            odd = odd.next
            even.next = odd.next
            even = even.next
        odd.next = evenHead
        return head